# Environments

In [1]:
import os
import optax
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

import lofi_tabular as experiment

In [2]:
import jax
import jax.numpy as jnp
from bandits.environments.tabular_env import TabularEnvironment
from bandits.environments.mnist_env import MnistEnvironment

```python
from bandits.environments import SomeEnv
from bandits.agents.bayes_agent import BayesAgent
from bandits import training as btrain

env_kwargs = {...}
bandit_kwargs = {...}

env = SomeEnv(*env_kwargs)
bandit = BayesAgent(**bandit_kwargs)

key = jax.random.PRNGkey(...)
key_warm, key_run = jax.random.split(key)
bel_warmup, output_warmup = btrain.warmup_bandit(key_warmup, bandit, env, npulls)
bel_bandit, output_train = btrain.run_bandit(key_run, bandit, bel_warmup, env, t_init=npulls)
```

In [4]:
key = jax.random.PRNGKey(314)
n_train = 5000
name = "statlog"
path = "../bandit-data/"

env = TabularEnvironment(key, name, n_train, path=path)
type(env)

bandits.environments.environment.BanditEnvironment

# Eval

In [10]:
"""
In this demo, we evaluate the performance of the
lofi bandit on a tabular dataset
"""
import jax
import pickle
import numpy as np
import flax.linen as nn
from datetime import datetime
from bayes_opt import BayesianOptimization
from bandits import training as btrain
from bandits.agents.linear_bandit import LinearBandit
from bandits.agents.low_rank_filter_bandit import LowRankFilterBandit
from bandits.agents.neural_linear import NeuralLinearBandit

from bandits.environments.mnist_env import MnistEnvironment

import lofi_tabular as experiment

In [11]:
class MLP(nn.Module):
    num_arms: int

    @nn.compact
    def __call__(self, x):
        # x = nn.Dense(50)(x)
        # x = nn.relu(x)
        x = nn.Dense(500, name="last_layer")(x)
        x = nn.relu(x)
        x = nn.Dense(self.num_arms)(x)
        return x

In [6]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

In [80]:
ntrials = 1
npulls = 20
key = jax.random.PRNGKey(314)
key_env, key_train, key_eval = jax.random.split(key, 3)

ntrain = 500 # 5000
# env = TabularEnvironment(key_env, ntrain=ntrain, name='statlog', intercept=False, path="../bandit-data")
env = MnistEnvironment(key, ntrain=ntrain)
env_test = MnistEnvironment(key, ntrain=5000)

num_arms = env.labels_onehot.shape[-1]
model = MLP(num_arms)

In [81]:
optimisers = {}
results = {}

## LoFi

In [55]:
name_agent = "lofi"
extra_params = {
    "memory_size": 10,
    "model": model,
    # "dynamics_weights": 1.0,
    # "dynamics_covariance": 0.0,

}

@jax.jit
def bbf(**heval):
    res = experiment.warmup_and_run(
        heval, 
        experiment.transform_hparams_lofi,
        LowRankFilterBandit,
        env,
        key_train,
        npulls,
        **extra_params,
    )

    
    return res["hist_train"]["rewards"].sum()

In [56]:
%%time
# try q=0, γ=1
pbounds = {
    "log_em_cov": [-15, -1],
    "log_init_cov": [-15, -1],
    "log_1m_dweights": [-18, -6],
    "log_dcov": [-18, -6],

}


optimiser = BayesianOptimization(
    f=bbf,
    pbounds=pbounds,
    random_state=2718,
)

optimiser.maximize(init_points=5, n_iter=15)

|   iter    |  target   | log_1m... | log_dcov  | log_em... | log_in... |
-------------------------------------------------------------------------
| 1         | 139.0     | -12.89    | -9.565    | -3.12     | -14.94    |
| 2         | 473.0     | -16.99    | -17.91    | -7.946    | -8.266    |
| 3         | 96.0      | -7.491    | -7.542    | -1.492    | -1.936    |
| 4         | 418.0     | -17.55    | -16.42    | -6.22     | -6.497    |
| 5         | 441.0     | -7.93     | -9.276    | -7.165    | -10.8     |
| 6         | 431.0     | -12.77    | -15.46    | -9.428    | -8.35     |
| 7         | 97.0      | -17.92    | -18.0     | -13.66    | -6.249    |
| 8         | 453.0     | -14.02    | -17.77    | -5.883    | -9.1      |
| 9         | 287.0     | -18.0     | -18.0     | -6.072    | -12.38    |
| 10        | 463.0     | -8.08     | -13.98    | -7.81     | -9.528    |
| 11        | 536.0     | -7.061    | -11.21    | -12.04    | -10.06    |
| 12        | 437.0     | -6.0      | 

In [57]:
%%time
res = experiment.warmup_and_run(
    optimiser.max["params"], 
    experiment.transform_hparams_lofi,
    LowRankFilterBandit,
    env,
    key_eval,
    npulls,
    n_trials=8,
    **extra_params,
)

res = jax.tree_map(np.array, res)
results[name_agent] = res
optimisers[name_agent] = optimiser

CPU times: user 29.3 s, sys: 950 ms, total: 30.2 s
Wall time: 10.3 s


## LoFi: Fixed dynamics

In [82]:
name_agent = "lofi-fixed"
extra_params = {
    "memory_size": 10,
    "model": model,
    "dynamics_weights": 1.0,
    "dynamics_covariance": 0.0,

}

@jax.jit
def bbf(**heval):
    res = experiment.warmup_and_run(
        heval, 
        experiment.transform_hparams_lofi_fixed,
        LowRankFilterBandit,
        env,
        key_train,
        npulls,
        **extra_params,
    )

    
    return res["hist_train"]["rewards"].sum()

In [83]:
%%time
# try q=0, γ=1
pbounds = {
    "log_em_cov": [-15, -1],
    "log_init_cov": [-15, -1],
}


optimiser = BayesianOptimization(
    f=bbf,
    pbounds=pbounds,
    random_state=2718,
)

optimiser.maximize(init_points=5, n_iter=15)

|   iter    |  target   | log_em... | log_in... |
-------------------------------------------------
| 1         | 164.0     | -9.033    | -5.159    |
| 2         | 80.0      | -3.12     | -14.94    |
| 3         | 228.0     | -13.82    | -14.9     |
| 4         | 193.0     | -7.946    | -8.266    |
| 5         | 53.0      | -2.74     | -2.798    |
| 6         | 193.0     | -13.0     | -10.62    |
| 7         | 167.0     | -10.26    | -14.97    |
| 8         | 45.0      | -15.0     | -1.0      |
| 9         | 211.0     | -15.0     | -12.99    |
| 10        | 214.0     | -14.92    | -14.84    |
| 11        | 46.0      | -14.97    | -7.96     |
| 12        | 199.0     | -10.08    | -11.32    |
| 13        | 212.0     | -12.59    | -13.04    |
| 14        | 86.0      | -1.0      | -9.239    |
| 15        | 216.0     | -10.3     | -8.671    |
| 16        | 56.0      | -8.46     | -1.0      |
| 17        | 214.0     | -13.9     | -13.89    |
| 18        | 206.0     | -12.52    | -15.0     |


In [85]:
%%time
res = experiment.warmup_and_run(
    optimiser.max["params"], 
    experiment.transform_hparams_lofi_fixed,
    LowRankFilterBandit,
    env_test,
    key_eval,
    npulls,
    n_trials=8,
    **extra_params,
)

res = jax.tree_map(np.array, res)
results[name_agent] = res
optimisers[name_agent] = optimiser

CPU times: user 29.5 s, sys: 733 ms, total: 30.3 s
Wall time: 26.9 s


## Linear

In [86]:
name_agent = "linear"

@jax.jit
def bbf(**heval):
    res = experiment.warmup_and_run(
        heval, 
        experiment.transform_hparams_linear,
        LinearBandit,
        env,
        key_train,
        npulls,
    )

    
    return res["hist_train"]["rewards"].sum()

In [87]:
%%time
pbounds = {
    "eta": [0, 10],
    "log_lambda": [-8, 0],
}

optimiser = BayesianOptimization(
    f=bbf,
    pbounds=pbounds,
    random_state=2718,
)

optimiser.maximize(init_points=5, n_iter=15)

|   iter    |  target   |    eta    | log_la... |
-------------------------------------------------
| 1         | 48.0      | 4.262     | -2.377    |
| 2         | 45.0      | 8.486     | -7.964    |
| 3         | 45.0      | 0.8413    | -7.942    |
| 4         | 40.0      | 5.039     | -4.152    |
| 5         | 52.0      | 8.757     | -1.028    |
| 6         | 45.0      | 3.228     | -6.277    |
| 7         | 57.0      | 5.14      | -0.7948   |
| 8         | 64.0      | 6.432     | 0.0       |
| 9         | 92.0      | 0.7073    | 0.0       |
| 10        | 191.0     | 0.005823  | -1.728    |
| 11        | 44.0      | 0.0       | -2.84     |
| 12        | 190.0     | 0.001661  | -1.662    |
| 13        | 60.0      | 0.4888    | -1.671    |
| 14        | 43.0      | 5.762     | -4.201    |
| 15        | 193.0     | 0.005683  | -1.998    |
| 16        | 196.0     | 0.001958  | -1.133    |
| 17        | 182.0     | 0.02616   | -0.6817   |
| 18        | 43.0      | 10.0      | -4.53     |


In [88]:
%%time
res = experiment.warmup_and_run(
    optimiser.max["params"], 
    experiment.transform_hparams_linear,
    LinearBandit,
    env_test,
    key_eval,
    npulls,
    n_trials=8,
)

results[name_agent] = res
optimisers[name_agent] = optimiser

CPU times: user 58.6 s, sys: 9.17 s, total: 1min 7s
Wall time: 10.3 s


## Neural linear

In [89]:
name_agent = "neural-linear"
extra_params = {
    "model": model,

}

@jax.jit
def bbf(**heval):
    res = experiment.warmup_and_run(
        heval, 
        experiment.transform_hparams_neural_linear,
        NeuralLinearBandit,
        env,
        key_train,
        npulls,
        **extra_params
    )
    
    return res["hist_train"]["rewards"].sum()

In [90]:
%%time
pbounds = {
    "eta": [0, 10],
    "log_lambda": [-8, 0],
    "log_lr": [-10, -2]
}


optimiser = BayesianOptimization(
    f=bbf,
    pbounds=pbounds,
    random_state=2718,
)

optimiser.maximize(init_points=5, n_iter=15)

|   iter    |  target   |    eta    | log_la... |  log_lr   |
-------------------------------------------------------------
| 1         | 45.0      | 4.262     | -2.377    | -3.211    |
| 2         | 53.0      | 0.04448   | -7.327    | -9.942    |
| 3         | 45.0      | 5.039     | -4.152    | -2.994    |
| 4         | 45.0      | 8.715     | -0.2811   | -2.535    |
| 5         | 45.0      | 0.375     | -6.947    | -4.983    |
| 6         | 40.0      | 2.44      | -8.0      | -10.0     |
| 7         | 45.0      | 5.03      | -4.264    | -4.832    |
| 8         | 57.0      | 9.86      | -1.025    | -7.315    |
| 9         | 49.0      | 9.911     | -1.267    | -7.7      |
| 10        | 46.0      | 9.907     | -0.8264   | -7.41     |
| 11        | 54.0      | 7.318     | -1.496    | -7.878    |
| 12        | 45.0      | 8.662     | -6.697    | -4.953    |
| 13        | 43.0      | 0.3076    | -2.579    | -5.038    |
| 14        | 45.0      | 3.857     | -6.982    | -6.741    |
| 15    

In [91]:
%%time
res = experiment.warmup_and_run(
    optimiser.max["params"], 
    experiment.transform_hparams_neural_linear,
    NeuralLinearBandit,
    env_test,
    key_eval,
    npulls,
    n_trials=8,
    **extra_params
)

results[name_agent] = res
optimisers[name_agent] = optimiser

CPU times: user 31 s, sys: 2.8 s, total: 33.8 s
Wall time: 8.91 s


## Plot results

linear - 3005.88±51.69
neural-linear - 1702.12±28.36
lofi - 3761.75±162.61


Text(0.5, 1.0, 'MNIST environment')

In [109]:
for name, res in results.items():
    rewards = res["hist_train"]["rewards"]
    mean_reward = rewards.sum(axis=1).mean()
    std_reward = rewards.sum(axis=1).std()

    rewards = pd.DataFrame(rewards).T.cumsum(axis=0)
    print(f"{name} - {mean_reward:0.2f}±{std_reward:0.2f}")

    nsteps = len(rewards)
    rewards.mean(axis=1).plot(label=name)
plt.plot([0, nsteps], [0, nsteps], c="tab:gray", linewidth=1, linestyle="--")
plt.legend()
plt.grid(alpha=0.3)
plt.ylabel("Cumulative reward")
plt.xlabel("Num. observations")
plt.title("MNIST environment")